# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [130]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Create list of filepaths to process original event csv data files

In [131]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/Users/angelica/Desktop/Sparkify_Data_Modeling_Cassandra
['/Users/angelica/Desktop/Sparkify_Data_Modeling_Cassandra/event_data/2018-11-09-events.csv', '/Users/angelica/Desktop/Sparkify_Data_Modeling_Cassandra/event_data/2018-11-04-events.csv', '/Users/angelica/Desktop/Sparkify_Data_Modeling_Cassandra/event_data/2018-11-01-events.csv', '/Users/angelica/Desktop/Sparkify_Data_Modeling_Cassandra/event_data/2018-11-10-events.csv', '/Users/angelica/Desktop/Sparkify_Data_Modeling_Cassandra/event_data/2018-11-06-events.csv', '/Users/angelica/Desktop/Sparkify_Data_Modeling_Cassandra/event_data/2018-11-03-events.csv', '/Users/angelica/Desktop/Sparkify_Data_Modeling_Cassandra/event_data/2018-11-08-events.csv', '/Users/angelica/Desktop/Sparkify_Data_Modeling_Cassandra/event_data/2018-11-05-events.csv', '/Users/angelica/Desktop/Sparkify_Data_Modeling_Cassandra/event_data/2018-11-02-events.csv', '/Users/angelica/Desktop/Sparkify_Data_Modeling_Cassandra/event_data/2018-11-07-events.csv']


#### Process files to create the data file csv that will be used for Apache Casssandra tables

In [132]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
#             print(line)
            full_data_rows_list.append(line) 
            
# total number of rows 
# print(len(full_data_rows_list))
# list of event data rows
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [133]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

1595


# Part II. Apache Cassandra 

## The CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

<img src="images/image_event_datafile_new.jpg">

## Apache Cassandra code

#### Creating a Cluster

In [134]:
# Make a connection to a Cassandra instance in local machine 
# (127.0.0.1)

try:
    from cassandra.cluster import Cluster
    cluster = Cluster()
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [135]:
# Create Keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION =
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [136]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data:

### 1. Output  the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [137]:
## 1:  Output artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query = "CREATE TABLE IF NOT EXISTS session "
query = query + "(sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY (sessionId, itemInSession))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [138]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
# Assign INSERT statements into the `query` variable
        query = "INSERT INTO session (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### SELECT statement to verify that the data have been inserted into each table

In [139]:
# Run the SELECT statement to verify the data was entered into the table
query = "select artist, song, length FROM session WHERE sessionId=338 AND iteminsession=4"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

session_data = []

for row in rows:
    session_data.append((row.artist, row.song, row.length))
    
session_df = pd.DataFrame(session_data, columns=['Artist', 'Song', 'Length'])
session_df

,Artist,Song,Length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


### 2. Output name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [140]:
## 2: Output name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query = "CREATE TABLE IF NOT EXISTS user_session "
query = query + "(userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, PRIMARY KEY ((userId, sessionId), itemInSession))"

try:
    session.execute(query)
except Exception as e:
    print(e) 

In [141]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# Assign INSERT statements into the `query` variable
        query = "INSERT INTO user_session (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [142]:
# Run SELECT statement to verify the data was entered into the table
query = "select artist, song, firstName, lastName FROM user_session WHERE userId=10 AND sessionId=182"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

user_session_data = [] 

for row in rows:
    user_session_data.append((row.artist, row.song, row.firstname, row.lastname))
    
user_session_df = pd.DataFrame(user_session_data, columns=['Artist', 'Song', 'First_Name', 'Last_Name'])
user_session_df

,Artist,Song,First_Name,Last_Name
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


### 3. Output every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [143]:
## 3: Output every user name (first and last) in the music app history who listened to the song 'All Hands Against His Own'
query = "CREATE TABLE IF NOT EXISTS song_user "
query = query + "(song text, userId int, firstName text, lastName text, PRIMARY KEY (song, userId))"

try:
    session.execute(query)
except Exception as e:
    print(e)                  

In [144]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
# Assign INSERT statements into the `query` variable
        query = "INSERT INTO song_user (song, userId, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [145]:
# Run SELECT statement to verify the data was entered into the table
query = "select firstName, lastName FROM song_user WHERE song='All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

song_user_data = []

for row in rows:
    song_user_data.append((row.firstname, row.lastname))
    
song_user_df = pd.DataFrame(song_user_data, columns=['First_Name', 'Last_Name'])
song_user_df

,First_Name,Last_Name
0,Sara,Johnson


### Drop the tables before closing out the sessions

In [146]:
## Drop the table before closing out the sessions
session.execute("drop table if exists session")
session.execute("drop table if exists user_session")
session.execute("drop table if exists song_user")


### Close the session and cluster connection¶

In [147]:
session.shutdown()
cluster.shutdown()